In [ ]:
import osmnx as ox

In [ ]:
G = ox.graph_from_place("Paris, France", network_type="walk")

In [ ]:
from utils import display_network

display_network(G)

In [ ]:
G = ox.graph_from_place("Paris, France", network_type="drive")

In [ ]:
from utils import display_network

display_network(G)

In [ ]:
# https://github.com/matejker/everystreet

CUSTOM_FILTER = (
    '["highway"]["area"!~"yes"]["highway"!~"bridleway|bus_guideway|bus_stop|construction|'
    "cycleway|elevator|footway|motorway|motorway_junction|motorway_link|escalator|proposed|"
    'construction|platform|raceway|rest_area|path|service"]["access"!~"customers|no|private"]'
    '["public_transport"!~"platform"]["fee"!~"yes"]["foot"!~"no"]["service"!~"drive-through|'
    'driveway|parking_aisle"]["toll"!~"yes"]'
)

In [ ]:
G = ox.graph_from_place("Paris, France", custom_filter=SIMPLE_FILTER)

In [ ]:
from utils import display_network

display_network(G)

In [ ]:
streets = ox.graph_to_gdfs(G, nodes=False, edges=True)

In [ ]:
streets.head()

In [ ]:
print(f"NaN count: {streets['name'].isna().sum()}")
print(f"List count: {streets['name'].apply(lambda x: isinstance(x, (list))).sum()}")

In [ ]:
from utils import ensure_string

s = (streets.name.isna().cumsum() + 1).astype(int)
streets["name"] = streets["name"].fillna("Unnamed street " + s.astype(str))
streets["name"] = streets["name"].apply(ensure_string)

In [ ]:
print(f"NaN count: {streets['name'].isna().sum()}")
print(f"List count: {streets['name'].apply(lambda x: isinstance(x, (list))).sum()}")

In [ ]:
counts = streets['name'].value_counts()
duplicates = counts[counts > 1]
print(len(duplicates))

In [ ]:
unique_streets = streets.dissolve(by="name", as_index=False)
unique_streets = unique_streets.to_crs("EPSG:4326")

In [ ]:
counts = unique_streets['name'].value_counts()
duplicates = counts[counts > 1]
print(len(duplicates))

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import declarative_base, sessionmaker

DATABASE_URL = "postgresql://vinayak@localhost:5432/paris"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Float, Integer, String
from geoalchemy2 import Geometry

class Street(Base):
    __tablename__ = "streets"
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    geometry = Column(Geometry("MULTILINESTRING", srid=4326))

Base.metadata.create_all(engine)

In [ ]:
for _, row in unique_streets.iterrows():
    street = Street(name=row["name"], geometry="SRID=4326;" + row["geometry"].wkt)
    session.add(street)
    session.commit()

In [ ]:
streets = session.query(Street).all()
print(f"Found {len(streets)} streets in database")